# Data exploration 

---

Group name: O

---


## Introduction

*This section includes a short description of the data* 

## Setup

In [ ]:
import pandas as pd
import altair as alt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

alt.data_transformers.disable_max_rows()


## Data

## Import data

In [ ]:
DATA = "../data/external/nba_elo_latest.csv"

### Data structure

In [ ]:
df = pd.read_csv(DATA)
print(df)

df.info()

### Data corrections

In [5]:
df['score1'] = df['score1'].astype("integer")
df['score2'] = df['score2'].astype("integer")
#df['...'] = df['...'].astype("category")

In [ ]:
df(info)

## Exploratory data analysis